# Datos

In [34]:
import pandas as pd
import glob
import os

# ==============================
# 1️⃣ Ruta donde están tus CSV
# ==============================
ruta = "./data_entrenamiento/"  # <-- cambia por tu carpeta
archivos = glob.glob(os.path.join(ruta, "*.csv"))

# ==============================
# 2️⃣ Leer todos los CSV y procesar
# ==============================
dfs = []

for archivo in archivos:
    # Obtener ticker desde el nombre del archivo (sin extensión)
    ticker = os.path.splitext(os.path.basename(archivo))[0]
    
    # Leer CSV
    df = pd.read_csv(archivo)
    
    # Detectar columnas que empiecen con "Sector_"
    sector_cols = [c for c in df.columns if c.startswith("Sector_")]
    
    # Detectar el sector cuyo valor es 1
    sector_name = df.loc[0, sector_cols].idxmax().replace("Sector_", "")
    
    # Agregar columnas Ticker y Sector
    df["Ticker"] = ticker
    df["Sector"] = sector_name
    
    dfs.append(df)


In [35]:
# ==============================
# 3️⃣ Combinar todo en un solo DataFrame
# ==============================
final_df = pd.concat(dfs, ignore_index=True)


In [37]:
result = final_df[["Ticker", "Sector"]].copy()

In [39]:
# ==============================
# 2️⃣ Eliminar duplicados por Ticker
# ==============================
result = result.drop_duplicates(subset=["Ticker"])

In [41]:
# ==============================
# 3️⃣ Limpiar nombres de Ticker
# ==============================
result["Ticker"] = result["Ticker"].str.replace("_completo_arreglado", "", regex=False)

/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_39339/3061847798.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Ticker"] = result["Ticker"].str.replace("_completo_arreglado", "", regex=False)


In [42]:
result

,Ticker,Sector
0,GOOGL,communication-services
830,SRE,utilities
1660,STT,financial-services
2490,PGR,financial-services
3320,LEN,consumer-cyclical
...,...,...
414477,BIIB,healthcare
415307,FDS,financial-services
416222,ADM,consumer-defensive
417052,KMI,energy


In [44]:
# ==============================
# 4️⃣ Resultado final
# ==============================
print(result.head())

# (Opcional) Guardar a nuevo archivo
result.to_csv("dataset_limpio.csv", index=False)


     Ticker                  Sector
0     GOOGL  communication-services
830     SRE               utilities
1660    STT      financial-services
2490    PGR      financial-services
3320    LEN       consumer-cyclical


# COMPLETO

In [45]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np

# ==============================
# 1️⃣ Cargar el CSV
# ==============================
df = pd.read_csv("./V. SP500 Completo/resultados_predicciones_futuras_Completo.csv")

# ==============================
# 2️⃣ Calcular el RMSE solo para Pred_Average
# ==============================
# Real = Rentabilidad
rmse = np.sqrt(mean_squared_error(df["Rentabilidad"], df["Pred_Average"]))

print(rmse)


26.758841989367927


In [46]:
result = df[["Ticker", "Pred_Average", "Rentabilidad", "Dif_Average"]].copy()

In [47]:
result["Dif_Average"] = result["Dif_Average"].abs()

In [48]:
result

,Ticker,Pred_Average,Rentabilidad,Dif_Average
0,LULU,35.857423,-10.777444,46.634866
1,KVUE,32.305301,-33.328167,65.633468
2,CAG,32.113118,-8.866028,40.979145
3,LYB,32.001959,-5.266232,37.268191
4,PAYC,31.523755,-19.705873,51.229628
...,...,...,...,...
497,GOOGL,-28.174536,43.212649,71.387185
498,GS,-28.320531,10.261436,38.581967
499,LRCX,-29.240820,59.123488,88.364308
500,TER,-30.585872,70.902560,101.488432


In [49]:
result_sorted = result.sort_values(by="Dif_Average", ascending=True)

In [50]:
result_sorted

,Ticker,Pred_Average,Rentabilidad,Dif_Average
197,AIG,3.730648,3.686664,0.043983
253,DIS,-0.389462,-0.230328,0.159134
226,NXPI,1.777491,2.027076,0.249585
250,PHM,-0.141781,-0.500625,0.358845
184,PRU,5.012068,4.622280,0.389787
...,...,...,...,...
500,TER,-30.585872,70.902560,101.488432
437,WBD,-12.770463,89.291749,102.062212
461,INTC,-14.451664,102.276180,116.727844
483,WDC,-22.460670,102.021578,124.482248


In [51]:
import pandas as pd

df_sector = pd.read_csv("dataset_limpio.csv")        # contiene Ticker, Sector

# ==============================
# 2️⃣ Hacer merge por Ticker
# ==============================
df_final = pd.merge(result_sorted, df_sector, on="Ticker", how="left")

# ==============================
# 3️⃣ Ver resultado
# ==============================
print(df_final)


    Ticker  Pred_Average  Rentabilidad  Dif_Average                  Sector
0      AIG      3.730648      3.686664     0.043983      financial-services
1      DIS     -0.389462     -0.230328     0.159134  communication-services
2     NXPI      1.777491      2.027076     0.249585              technology
3      PHM     -0.141781     -0.500625     0.358845       consumer-cyclical
4      PRU      5.012068      4.622280     0.389787      financial-services
..     ...           ...           ...          ...                     ...
497    TER    -30.585872     70.902560   101.488432              technology
498    WBD    -12.770463     89.291749   102.062212  communication-services
499   INTC    -14.451664    102.276180   116.727844              technology
500    WDC    -22.460670    102.021578   124.482248              technology
501     MU    -33.334501    100.152083   133.486584              technology

[502 rows x 5 columns]


In [52]:
df_final.to_csv("diferencias_mse_Completo.csv", index=False)

# SIN TECNOLOGIA

In [77]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np

# ==============================
# 1️⃣ Cargar el CSV
# ==============================
df = pd.read_csv("./V. Sin Tecnologia/resultados_predicciones_futuras_sin_Tecnologia.csv")

# ==============================
# 2️⃣ Calcular el RMSE solo para Pred_Average
# ==============================
# Real = Rentabilidad
rmse = np.sqrt(mean_squared_error(df["Rentabilidad"], df["Pred_Average"]))

In [78]:
rmse

24.97516764623436

In [64]:
result = df[["Ticker", "Pred_Average", "Rentabilidad", "Dif_Average"]].copy()

In [65]:
result["Dif_Average"] = result["Dif_Average"].abs()

In [66]:
result

,Ticker,Pred_Average,Rentabilidad,Dif_Average
0,LULU,37.669966,-10.777444,48.447410
1,LYB,32.737279,-5.266232,38.003510
2,DOW,30.353355,11.960448,18.392907
3,FI,29.999311,-49.743780,79.743091
4,STZ,29.119318,-23.558974,52.678292
...,...,...,...,...
497,PLTR,-23.841510,10.027445,33.868955
498,MU,-27.140586,100.152083,127.292669
499,GOOG,-27.766916,42.980489,70.747406
500,GOOGL,-28.061998,43.212649,71.274647


In [67]:
result_sorted = result.sort_values(by="Dif_Average", ascending=True)

In [68]:
import pandas as pd

df_sector = pd.read_csv("dataset_limpio.csv")        # contiene Ticker, Sector

# ==============================
# 2️⃣ Hacer merge por Ticker
# ==============================
df_final = pd.merge(result_sorted, df_sector, on="Ticker", how="left")

# ==============================
# 3️⃣ Ver resultado
# ==============================
print(df_final)

    Ticker  Pred_Average  Rentabilidad  Dif_Average                  Sector
0      BLK     -2.343303     -2.238241     0.105062      financial-services
1     PYPL      1.665088      1.539131     0.125956      financial-services
2      APA     17.914691     17.695700     0.218992                  energy
3      BXP      9.165714      9.388612     0.222899             real-estate
4     NXPI      1.778679      2.027076     0.248397              technology
..     ...           ...           ...          ...                     ...
497    TER    -22.535135     70.902560    93.437695              technology
498    WBD     -7.260236     89.291749    96.551984  communication-services
499    WDC    -12.360414    102.021578   114.381992              technology
500   INTC    -13.381115    102.276180   115.657294              technology
501     MU    -27.140586    100.152083   127.292669              technology

[502 rows x 5 columns]


In [70]:
df_final.to_csv("diferencias_mse_Sin_Tecnologia.csv", index=False)

# REVISAR RMSE SIN TECNOLOGIA

In [71]:
df_final

,Ticker,Pred_Average,Rentabilidad,Dif_Average,Sector
0,BLK,-2.343303,-2.238241,0.105062,financial-services
1,PYPL,1.665088,1.539131,0.125956,financial-services
2,APA,17.914691,17.695700,0.218992,energy
3,BXP,9.165714,9.388612,0.222899,real-estate
4,NXPI,1.778679,2.027076,0.248397,technology
...,...,...,...,...,...
497,TER,-22.535135,70.902560,93.437695,technology
498,WBD,-7.260236,89.291749,96.551984,communication-services
499,WDC,-12.360414,102.021578,114.381992,technology
500,INTC,-13.381115,102.276180,115.657294,technology


In [82]:
import pandas as pd

# Supongamos que ya tienes tu DataFrame cargado como df

# ==============================
# 1️⃣ Quitar todos los del sector "technology"
# ==============================
df_data = df_final[df_final["Sector"] != "technology"]
df_data = df_data[df_data["Sector"] != None]

In [83]:
# ==============================
# 2️⃣ Eliminar tickers específicos
# ==============================
tickers_a_eliminar = ["GOOGL", "GOOG", "AMZN", "META", "TSLA"]  # agrega los que quieras
df_data = df_data[~df_data["Ticker"].isin(tickers_a_eliminar)]

# ==============================
# 3️⃣ Mostrar resultado
# ==============================
print(df_data)


    Ticker  Pred_Average  Rentabilidad  Dif_Average                  Sector
0      BLK     -2.343303     -2.238241     0.105062      financial-services
1     PYPL      1.665088      1.539131     0.125956      financial-services
2      APA     17.914691     17.695700     0.218992                  energy
3      BXP      9.165714      9.388612     0.222899             real-estate
5     RVTY      7.908184      8.327253     0.419070              healthcare
..     ...           ...           ...          ...                     ...
485   KVUE     22.365740    -33.328167    55.693907      consumer-defensive
486    FDS     25.175285    -30.798026    55.973311      financial-services
488    TTD     15.278125    -43.077100    58.355225  communication-services
491    APP    -18.548094     45.728729    64.276822                     NaN
498    WBD     -7.260236     89.291749    96.551984  communication-services

[416 rows x 5 columns]


In [84]:
rmse = np.sqrt(mean_squared_error(df_data["Rentabilidad"], df_data["Pred_Average"]))

In [85]:
rmse

20.90378841680041

# RMSE por SECTORES

RMSE ENTRENAMIENTO COMPLETO

In [88]:
import pandas as pd
import numpy as np

# Cargar CSV
df = pd.read_csv("./V. SP500 Completo/diferencias_mse_Completo.csv")

# Calcular RMSE por sector
rmse_por_sector_completo = (
    df.groupby("Sector")
      .apply(lambda g: np.sqrt(np.mean((g["Pred_Average"] - g["Rentabilidad"])**2)))
      .reset_index(name="RMSE")
)

print(rmse_por_sector_completo)


                    Sector       RMSE
0          basic-materials  24.836167
1   communication-services  37.804231
2        consumer-cyclical  23.234383
3       consumer-defensive  25.783526
4                   energy  20.159555
5       financial-services  18.658365
6               healthcare  23.152368
7              industrials  20.845860
8              real-estate  19.553449
9               technology  40.848383
10               utilities  15.282074


/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_39339/3588641715.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: np.sqrt(np.mean((g["Pred_Average"] - g["Rentabilidad"])**2)))


RMSE ENTRENAMIENTO SIN TECNOLOGIA

In [89]:
import pandas as pd
import numpy as np

# Cargar CSV
df = pd.read_csv("./V. Sin Tecnologia/diferencias_mse_Sin_Tecnologia.csv")

# Calcular RMSE por sector
rmse_por_sector_sin_Tech = (
    df.groupby("Sector")
      .apply(lambda g: np.sqrt(np.mean((g["Pred_Average"] - g["Rentabilidad"])**2)))
      .reset_index(name="RMSE")
)

print(rmse_por_sector_sin_Tech)


                    Sector       RMSE
0          basic-materials  23.598339
1   communication-services  36.906222
2        consumer-cyclical  21.428216
3       consumer-defensive  22.622007
4                   energy  17.612718
5       financial-services  18.283008
6               healthcare  21.993873
7              industrials  19.595090
8              real-estate  19.218856
9               technology  38.064950
10               utilities  12.549327


/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_39339/3968832276.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: np.sqrt(np.mean((g["Pred_Average"] - g["Rentabilidad"])**2)))
